In [1]:
!pip install -q pybotters

## 必要なこと残り
* 執行系
  * tuned_qty
  * 執行時間(特にclose)

In [129]:
import asyncio
import pybotters
from dotenv import load_dotenv
import os
import numpy as np
import datetime

load_dotenv('.env')
_

_  = '=============================='
pair_symbol = 'BLZUSDT'
volume = 10



async with pybotters.Client(apis=apis, base_url='https://api-testnet.bybit.com') as client:
    print(f'{_} set-leverage {_}')
    async with client.post(
        "/v5/position/set-leverage",
        data={'category': 'linear', 'symbol': 'TRBUSDT', 'buyLeverage': '5', 'sellLeverage': '5'}) as resp:
        content = await resp.json()
        pprint(content['retMsg'])

    price = None
    async with client.post(
        "/v5/order/create",
        data={
            "category": "linear",
            "symbol": "BTCUSDT",
            "side": "Buy",
            'orderType': 'Limit' if  price else 'Market', 
            "qty": "0.001",
            'reduceOnly': False,
        }) as resp:
        content = await resp.json()
        pprint(content)

    print(f'{_} position {_}')
    async with client.get("/v5/position/list?category=linear&settleCoin=USDT") as resp:
        content = await resp.json()
        symbol_list = [{'symbol': json['symbol'], 'side': json['side'], 'size': json['size']} for json in content['result']['list']]
        pprint(symbol_list)
      
    print(f'{_} funding info {_}')
    async with client.get(f"/v5/market/tickers?category=linear&symbol={pair_symbol}") as resp:
        content = await resp.json()
        result = content['result']['list'][0]
        pprint([result['symbol'], result['fundingRate'], result['nextFundingTime']])
        before_1h_next_funding = int(result['nextFundingTime']) - 1 * 60 * 60 * 1000
        pprint(f'before_1h_next_funding: {before_1h_next_funding}')

    print(f'{_} entry signal {_}')
    async with client.get(
        f"/v5/market/kline?category=linear&interval=60&symbol={pair_symbol}&end={before_1h_next_funding - 1}&limit=6"
    ) as resp:
        content = await resp.json()
        returns = [(float(ohlcv[3]) - float(ohlcv[0])) / float(ohlcv[0]) * 100 for ts, *ohlcv, vol, turnover in content['result']['list']]
#         pprint(returns)
        return_2h_before = returns[0]
        return_from_7h_to_2h = sum(returns)
#         print(return_from_7h_to_2h, return_2h_before)
        
        entry_signal = return_from_7h_to_2h < - 0.141 and return_2h_before < 0
        pprint(entry_signal)

    print(f'{_} precision info {_}')
    async with client.get(
        f"/v5/market/instruments-info?category=linear&symbol={pair_symbol}"
    ) as resp:
        content = await resp.json()
        result = content['result']['list'][0]
#         pprint(result)
        
        pprint(result['leverageFilter'] | result['priceFilter'] | result['lotSizeFilter'])

    print(f'{_} latest price {_}')
    async with client.get(
        f"/v5/market/mark-price-kline?category=linear&symbol={pair_symbol}&interval=1"
    ) as resp:
        content = await resp.json()
        result = content['result']['list'][0]
        pprint(result)


============================== set-leverage ==============================
'OK'
{'result': {'orderId': '3bd5270f-ce61-47bf-bd2b-a23d313695f2',
            'orderLinkId': ''},
 'retCode': 0,
 'retExtInfo': {},
 'retMsg': 'OK',
 'time': 1694523835200}
============================== position ==============================
[{'side': 'Buy', 'size': '0.001', 'symbol': 'BTCUSDT'}]
============================== funding info ==============================
['BLZUSDT', '-0.007455', '1694534400000']
'before_1h_next_funding: 1694530800000'
============================== entry signal ==============================
False
============================== precision info ==============================
{'leverageStep': '0.01',
 'maxLeverage': '25.00',
 'maxOrderQty': '147750',
 'maxPrice': '19.99998',
 'minLeverage': '1',
 'minOrderQty': '1',
 'minPrice': '0.00001',
 'postOnlyMaxOrderQty': '1477500',
 'qtyStep': '1',
 'tickSize': '0.00001'}
============================== latest price =====================

In [67]:
print(result['nextFundingTime'])
ts = int(result['nextFundingTime']) / 1000
datetime.datetime.fromtimestamp(ts), datetime.datetime.now()

1694505600000


(datetime.datetime(2023, 9, 12, 8, 0),
 datetime.datetime(2023, 9, 12, 7, 23, 27, 299502))

## クラス定義

In [119]:
import asyncio
import pybotters
from dotenv import load_dotenv
import os
from pprint import pprint

class Exchange:
    def __init__(self, client):
        self.client = client

    async def get_symbol_info(self, pair_symbol):
        async with self.client.get(
            f"/v5/market/instruments-info?category=linear&symbol={pair_symbol}"
        ) as resp:
            content = await resp.json()
            result = content['result']['list'][0]
            symbol_info = result['leverageFilter'] | result['priceFilter'] | result['lotSizeFilter']
            return symbol_info

    async def get_funding_time(self, pair_symbol):
        async with self.client.get(f"/v5/market/tickers?category=linear&symbol={pair_symbol}") as resp:
            content = await resp.json()
            result = content['result']['list'][0]
            next_funding_time = int(result['nextFundingTime'])
            return next_funding_time
            
    async def get_latest_price(self, pair_symbol):
        async with self.client.get(
            f"/v5/market/mark-price-kline?category=linear&symbol={pair_symbol}&interval=1"
        ) as resp:
            content = await resp.json()
            ohlc = content['result']['list'][0]
            return float(ohlc[-1])

    async def get_latest_returns(self, pair_symbol, next_funding_time):
        before_1h_next_funding = next_funding_time - 1 * 60 * 60 * 1000
        async with self.client.get(
            f"/v5/market/kline?category=linear&interval=60&symbol={pair_symbol}&end={before_1h_next_funding - 1}&limit=6"
        ) as resp:
            content = await resp.json()
            returns = [(float(ohlcv[3]) - float(ohlcv[0])) / float(ohlcv[0]) * 100 for ts, *ohlcv, vol, turnover in content['result']['list']]
            return returns

    async def get_position(self):
        async with self.client.get("/v5/position/list?category=linear&settleCoin=USDT") as resp:
            content = await resp.json()
            symbol_list = [{'symbol': json['symbol'], 'side': json['side'], 'size': json['size']} for json in content['result']['list']]
            return symbol_list

    async def set_leverage(self, pair_symbol, leverage):
        async with self.client.post(
            "/v5/position/set-leverage",
            data={        
                'category': 'linear',
                'symbol': pair_symbol,
                'buyLeverage': leverage,
                'sellLeverage': leverage,
            }
        ) as resp:
            content = await resp.json()
            return content['retMsg']

    async def create_order(self, pair_symbol, qty, side='Buy', price=None, reduce_only=False):
        async with self.client.post(
            "/v5/order/create",
            data={
                'category': "linear",
                'symbol': pair_symbol,
                'side': side,
                'orderType': 'Limit' if  price else 'Market', 
                'qty': qty,
                'reduceOnly': reduce_only,
            }) as resp:
            content = await resp.json()
            return content['retMsg']


In [121]:
import datetime

class TradingManager:
    def __init__(self, apis, volume, pair_symbol=None, leverage=5):
        self.apis = apis
        self.volume = volume
        self.pair_symbol = pair_symbol
        self.leverage = leverage
        self.next_funding_time = None
        self.client = None
        
    async def open_position_strategy(self):
        async with pybotters.Client(apis=self.apis, base_url='https://api-testnet.bybit.com') as client:
            self.exchange = Exchange(client)
            
            if not await self.regression_chance_is_coming():
                return

            if not await self.signal_on():
                return
            
            await self.open_position()

            return

    async def close_position_strategy(self):
        async with pybotters.Client(apis=self.apis, base_url='https://api-testnet.bybit.com') as client:
            self.exchange.client = client
            
            if not await self.funding_is_coming():
                return

            symbols = await self.get_holding_symbols()
            for symbol in symbols:
                await self.close_position(symbol)

            return

    async def regression_chance_is_coming(self):
        """
        check_timing_for_position_opening
        """
        next_funding_timestamp = await self.exchange.get_funding_time(self.pair_symbol)
        next_funding_time = datetime.datetime.fromtimestamp(next_funding_timestamp / 1000)
        funding_time_is_coming = next_funding_time - datetime.datetime.now() < datetime.timedelta(minutes=60 + 1)
        chance_has_passed = next_funding_time - datetime.datetime.now() < datetime.timedelta(minutes=60)
        return funding_time_is_coming and (not chance_has_passed)

    async def funding_is_coming(self):
        """
        check_timing_for_position_closing
        """
        next_funding_timestamp = await self.exchange.get_funding_time(self.pair_symbol)
        next_funding_time = datetime.datetime.fromtimestamp(next_funding_timestamp / 1000)
        funding_time_is_coming = next_funding_time - datetime.datetime.now() < datetime.timedelta(minutes=1)
        funding_has_passed = next_funding_time < datetime.datetime.now()
        return funding_time_is_coming and (not funding_has_passed)


    async def signal_on(self):
        """"""
        returns = await self.exchange.get_latest_returns(self.pair_symbol, self.next_funding_time)
        
        return_2h_before = returns[0]
        return_from_7h_to_2h = sum(returns)
        entry_signal = return_from_7h_to_2h < - 0.141 and return_2h_before < 0
        return entry_signal
    
    async def get_holding_symbols(self):
        """
        simplifing now
        """
        symbol_list = await self.exchange.get_position()
        symbols = [dic['symbol'] for dic in symbol_list]
        return symbols
        

    async def open_position(self):
        """
        implement limit order in the future
        """
        response = await self.exchange.set_leverage(self.pair_symbol, self.leverage)
        if response != 'OK' and response != 'leverage not modified':
            pass  # should raise error

        tuned_qty = await self.get_tuned_qty()
        response = await self.exchange.create_order(self.pair_symbol, tuned_qty, side='Buy', price=None, reduce_only=False)
        if response != 'OK':
            pass  # should raise error
        return
    
    async def close_position(self, pair_symbol):
        """
        implement limit order in the future
        """
        max_qty = await self.get_max_qty()
        response = await self.exchange.create_order(pair_symbol, max_qty, side='Sell', price=None, reduce_only=True)
        if response != 'OK':
            pass  # should raise error
        return
    
    async def get_tuned_qty(self):
        """"""
        latest_price = await self.exchange.get_latest_price(self.pair_symbol)
        qty = self.volume / latest_price

        symbol_info = await self.exchange.get_symbol_info(self.pair_symbol)
        qty_precision = symbol_info['qtyStep']
        max_qty = symbol_info['maxOrderQty']

        # qty tuning
        if qty_precision // 1 == qty_precision:
            qty_precision = int(qty_precision)

        tuned_qty = round(quantity / qty_precision) * qty_precision
        return min(tuned_qty, max_qty)
    
    async def get_max_qty(self):
        """"""
        symbol_info = await self.exchange.get_symbol_info(self.pair_symbol)
        max_qty = symbol_info['maxOrderQty']
        return max_qty
    

In [101]:
load_dotenv('.env')
apis = {
    'bybit_testnet': [os.getenv('KEY'), os.getenv('SECRET')]
}
pair_symbol = 'BLZUSDT'
volume = 10

bot = BybitBot(apis, pair_symbol, volume)
asyncio.run(bot.run())

RuntimeError: asyncio.run() cannot be called from a running event loop